# Default notebook

This default notebook is executed using a Lakeflow job as defined in resources/sample_job.job.yml.

In [0]:
%pip install databricks-vectorsearch flashrank databricks_langchain
dbutils.library.restartPython()

In [0]:
# from databricks.vector_search.client import VectorSearchClient
# from databricks.sdk import WorkspaceClient
# from databricks_langchain import DatabricksVectorSearch, DatabricksEmbeddings
# import databricks.sdk.service.catalog as c
# from mlflow.deployments import get_deploy_client
# from flashrank import Ranker, RerankRequest


# vsc = VectorSearchClient(disable_notice=True)
# ranker = Ranker(
#     model_name="rank-T5-flan", cache_dir="/Volumes/dev_appian_poc/00_bronze/artifacts"
# )


# embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

# if not vsc.endpoint_exists("appian_vsc_poc"):
#     vsc.create_endpoint(name="appian_vsc_poc")


# if not vsc.index_exists(
#     endpoint_name="appian_vsc_poc",
#     index_name="dev_appian_poc.02_gold.ingested_text_index",
# ):
#     index = vsc.create_delta_sync_index(
#         endpoint_name="appian_vsc_poc",
#         source_table_name="dev_appian_poc.02_gold.ingestion_text_embeddings",
#         index_name="dev_appian_poc.02_gold.ingested_text_index",
#         pipeline_type="TRIGGERED",
#         primary_key="id",
#         embedding_vector_column="embedding",
#         embedding_dimension=1024,
#         embedding_model_endpoint_name="databricks-bge-large-en",
#     )
# else:
#     index = vsc.get_index(
#         endpoint_name="appian_vsc_poc",
#         index_name="dev_appian_poc.02_gold.ingested_text_index",
#     )


# vector_store = DatabricksVectorSearch(
#     index_name="dev_appian_poc.02_gold.ingested_text_index",
#     text_column="content",
#     embedding=embedding_model,
# )

# sim = vector_store.as_retriever(search_kwargs={"k": 2}).invoke("who is robert peng")

# print(sim)

In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
from databricks_langchain import DatabricksVectorSearch, DatabricksEmbeddings
import databricks.sdk.service.catalog as c
from mlflow.deployments import get_deploy_client
from flashrank import Ranker, RerankRequest


def get_retriever(persist_dir: str = "/Volumes/dev_appian_poc/00_bronze/artifacts"):
    vsc = VectorSearchClient(disable_notice=True)
    if not vsc.index_exists(
        endpoint_name="appian_vsc_poc",
        index_name="dev_appian_poc.02_gold.ingested_text_index",
    ):
        index = vsc.create_delta_sync_index(
            endpoint_name="appian_vsc_poc",
            source_table_name="dev_appian_poc.02_gold.ingestion_text_embeddings",
            index_name="dev_appian_poc.02_gold.ingested_text_index",
            pipeline_type="TRIGGERED",
            primary_key="id",
            embedding_vector_column="embedding",
            embedding_dimension=1024,
            embedding_model_endpoint_name="databricks-bge-large-en",
        )
    else:
        index = vsc.get_index(
            endpoint_name="appian_vsc_poc",
            index_name="dev_appian_poc.02_gold.ingested_text_index",
        )

    vector_store = DatabricksVectorSearch(
        index_name="dev_appian_poc.02_gold.ingested_text_index",
        text_column="content",
        embedding=embedding_model,
        )
    return vector_store.as_retriever(search_kwargs={"k": 2})
    

In [0]:
vs = get_retriever()
similar_documents = vs.invoke("who is robert peng")
print(similar_documents)

In [0]:
results_arr = [row[1] for row in results['result']['data_array']]
passages_dicts = [{"text": passage} for passage in results_arr]

rerankrequest = RerankRequest(
    query=query_text,
    passages=passages_dicts
)

rerankresponse = ranker.rerank(rerankrequest)
